# Imports:

In [1]:
!pip3 install mediapipe opencv-python

  Obtaining dependency information for mediapipe from https://files.pythonhosted.org/packages/e9/7b/cd671c5067a56e1b4a9b70d0e42ac8cdb9f63acdc186589827cf213802a5/mediapipe-0.10.9-cp311-cp311-win_amd64.whl.metadata
  Using cached mediapipe-0.10.9-cp311-cp311-win_amd64.whl.metadata (9.8 kB)
  Obtaining dependency information for opencv-python from https://files.pythonhosted.org/packages/38/d2/3e8c13ffc37ca5ebc6f382b242b44acb43eb489042e1728407ac3904e72f/opencv_python-4.8.1.78-cp37-abi3-win_amd64.whl.metadata
  Using cached opencv_python-4.8.1.78-cp37-abi3-win_amd64.whl.metadata (20 kB)
  Obtaining dependency information for absl-py from https://files.pythonhosted.org/packages/01/e4/dc0a1dcc4e74e08d7abedab278c795eef54a224363bb18f5692f416d834f/absl_py-2.0.0-py3-none-any.whl.metadata
  Obtaining dependency information for flatbuffers>=2.0 from https://files.pythonhosted.org/packages/6f/12/d5c79ee252793ffe845d58a913197bfa02ae9a0b5c9bc3dc4b58d477b9e7/flatbuffers-23.5.26-py2.py3-none-any.whl.met

# Code: 

### Imports

In [2]:
import cv2
import mediapipe as mp 
import numpy as np 
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

### Calculations

##### Calculate Angle

In [3]:
def calculate_angle(a,b,c):
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)
    
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)
    
    if angle >180.0:
        angle = 360-angle
        
    return angle 

##### Get Leg Angles

In [4]:
def getAngles(landmarks):
    hip_left = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
    knee_left = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x,landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
    ankle_left = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]

    hip_right = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y]
    knee_right = [landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y]
    ankle_right = [landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y]

    angle_left = calculate_angle(hip_left, knee_left, ankle_left)
    angle_right = calculate_angle(hip_left, knee_left, ankle_right)

    return [angle_left, angle_right]

### Detections

In [5]:
# setup camera
cam = cv2.VideoCapture(0)

# squat counter variables
counter = 0

leg = "none"
flag = False

# debugging mode/display detections
debugMode = False


#setup mediapipe, 75%
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.75) as pose:
    while cam.isOpened():
        ret, frame = cam.read()
        
        # BRG -> RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
      
        results = pose.process(image)

        # Get landmarks
        try:
            landmarks = results.pose_landmarks.landmark
        except:
            pass
    
        # RGB -> BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        # get landmarks
        try:
            landmarks = results.pose_landmarks.landmark
            
            #get angles
            leftLeg, rightLeg = getAngles(landmarks)

            if leftLeg > 170 and rightLeg > 170:
                leg = "none"

            if leg == "right":
                if flag:
                    if rightLeg > 170:
                        flag = False
                        leg = "none"
                        counter += 1
                else:
                    if rightLeg < 70 and leftLeg > 170:
                        flag = True
            elif leg == "left":
                if flag:
                    if leftLeg > 170:
                        flag = False
                        leg = "none"
                        counter += 1
                else:
                    if leftLeg < 70 and rightLeg > 170:
                        flag = True
            else:
                if rightLeg < 170:
                    leg = "right"
                elif leftLeg < 170:
                    leg = "left"
            
            # show angle
            leftKnee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x, landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
            rightKnee = [landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y]
            cv2.putText(image, str(leftLeg), 
                        tuple(np.multiply(leftKnee, [frame.shape[0], frame.shape[1]]).astype(int)), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
            cv2.putText(image, str(rightLeg), 
                        tuple(np.multiply(rightKnee, [frame.shape[0], frame.shape[1]]).astype(int)), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
        except:
            pass
        
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                    mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
                                    mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2) 
                                    )
        
        # Rep data
        cv2.putText(image, 'REPS', (15,12), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
        cv2.putText(image, str(counter), 
                    (10,60), 
                    cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)
        
        cv2.imshow('Mediapipe Feed', image)

        # get input
        if cv2.waitKey(10) & 0xFF == ord('p'):
            break

    cam.release()
    cv2.destroyAllWindows()

### Clean Up:

In [6]:
cam.release()
cv2.destroyAllWindows()